# 🎨 Video Colorizer (Google Colab)

*Colorize black-and-white videos using DeOldify — all within Google Colab!*


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/cyamahat/DeOldify_Video_Colorizer/blob/main/DeOldify_Video_Colorizer_Colab.ipynb)

**Updated: November 2025**


---

This notebook is intended as a tool to colorize gifs and **short videos**. If you are trying to convert longer video, you may hit the limit on processing space.

#### **Credits**

Colab notebook adapted from DeOldify's official (and archived) source.

## Verify Correct Runtime Settings

**IMPORTANT**

In the "Runtime" menu for the notebook window, select "Change runtime type." Ensure that the following are selected:
* Runtime Type = Python 3
* Hardware Accelerator = GPU


## Git clone and install DeOldify

In [ ]:
!git clone https://github.com/jantic/DeOldify.git

In [ ]:
cd DeOldify

## Setup

In [ ]:
#NOTE:  This must be the first call in order to work properly!
from deoldify import device
from deoldify.device_id import DeviceId
#choices:  CPU, GPU0...GPU7
device.set(device=DeviceId.GPU0)

import torch

if not torch.cuda.is_available():
    print('GPU not available.')

from os import path

In [ ]:
!pip install -q -r requirements-colab.txt


In [ ]:
import sys, os
import warnings
from contextlib import redirect_stdout, redirect_stderr

print("📦 Importing libraries... please wait.")

with open(os.devnull, "w") as f, redirect_stdout(f), redirect_stderr(f):
    import fastai
    from deoldify.visualize import *
    from pathlib import Path
    import torch
    torch.backends.cudnn.benchmark = True
    warnings.filterwarnings("ignore", category=UserWarning, message=".*?Your .*? set is empty.*?")

print("✅ Libraries imported successfully.")


In [ ]:
!mkdir 'models'
!wget https://data.deepai.org/deoldify/ColorizeVideo_gen.pth -O ./models/ColorizeVideo_gen.pth


In [ ]:
import functools
import torch.serialization
from fastai.basic_train import Recorder
from fastai.callbacks.hooks import Hook
import fastai.callbacks.hooks as FCH
import torch.utils.hooks
import torch.nn.utils.spectral_norm

if not hasattr(torch, '_original_load_unpatched'):
    torch._original_load_unpatched = torch.load

def patched_torch_load(*args, **kwargs):
    current_patched_func_ref = torch.load
    torch.load = torch._original_load_unpatched

    try:
        if 'weights_only' not in kwargs:
            kwargs['weights_only'] = False
        return torch._original_load_unpatched(*args, **kwargs)
    finally:
        torch.load = patched_torch_load

if torch.load is not patched_torch_load:
    torch.load = patched_torch_load

colorizer = get_video_colorizer()

## Instructions

### *source_url*
Type in a url hosting a video from YouTube, Imgur, Twitter, Reddit, Vimeo, etc.  Many sources work!  GIFs also work.

**Recommendation:** To use your own video, upload it to an online storage service such as **Google Drive** and make sure the link is **publicly accessible** (anyone with the link can view). Then, use that link in the notebook.


### *render_factor*
The default value of 21 has been carefully chosen and should work -ok- for most scenarios (but probably won't be the -best-). This determines resolution at which the color portion of the video is rendered. Lower resolution will render faster, and colors also tend to look more vibrant. Older and lower quality film in particular will generally benefit by lowering the render factor. Higher render factors are often better for higher quality videos and inconsistencies (flashy render) will generally be reduced, but the colors may get slightly washed out.



### How to Download a Copy
Simply right click on the displayed video and click "Save video as..."!



### Pro Tips
1. If a video takes a long time to render and you're wondering how well the frames will actually be colorized, you can preview how well the frames will be rendered at each render_factor by using the code at the bottom. Just stop the video rendering by hitting the stop button on the cell, then run that bottom cell under "See how well render_factor values perform on a frame here". It's not perfect and you may still need to experiment a bit especially when it comes to figuring out how to reduce frame inconsistency.  But it'll go a long way in narrowing down what actually works.
2. If videos are taking way too much time for your liking, running the Jupyter notebook VideoColorizer.ipynb on your own machine (with DeOldify installed) will generally be much faster (as long as you have the hardware for it).   
3. Longer videos (running multiple minutes) are going to have a rough time on Colabs. You'll be much better off using a local install of DeOldify instead in this case.



### Troubleshooting
The video player may wind up not showing up, in which case- make sure to wait for the Jupyter cell to complete processing first (the play button will stop spinning).  Then follow these alternative download instructions

1. In the menu to the left, click Files
2. If you don't see the 'DeOldify' folder, click "Refresh"
3. By default, rendered video will be in /DeOldify/video/result/

If a video you downloaded doesn't play, it's probably because the cell didn't complete processing and the video is in a half-finished state.# New Section# New Section# New Section# New Section

## Colorize!

In [ ]:
source_url = 'https://raw.githubusercontent.com/cyamahat/DeOldify_Video_Colorizer/refs/heads/main/Lee_De_Forest_(1939).mp4' #@param {type:"string"}
render_factor = 21  #@param {type: "slider", min: 5, max: 40}
# watermarked = False #@param {type:"boolean"}
watermarked = False

if source_url is not None and source_url !='':
    video_path = colorizer.colorize_from_url(source_url, 'video.mp4', render_factor, watermarked=watermarked)
    show_video_in_notebook(video_path)
else:
    print('Provide a video url and try again.')

## See how well *render_factor* values perform on a frame here

In [ ]:
for i in range(10,40,2):
    colorizer.vis.plot_transformed_image('video/bwframes/video/00001.jpg', render_factor=i, display_render_factor=True, figsize=(8,8))